In [ ]:
from __future__ import division, print_function

import skimage
import numpy as np
from matplotlib import pyplot as plt

from scipy import fftpack

%matplotlib inline

In [ ]:
random_image = np.random.random([500, 500])

plt.imshow(random_image, cmap='gray', interpolation='nearest');

In [ ]:
from skimage import data
cat = data.chelsea()
print("Shape:", cat.shape)
print("Values min/max:", cat.min(), cat.max())

plt.imshow(cat, interpolation='nearest');

In [ ]:
linear0 = np.linspace(0, 1, 2500).reshape((50, 50))
linear1 = np.linspace(0, 255, 2500).reshape((50, 50)).astype(np.uint8)

print("Linear0:", linear0.dtype, linear0.min(), linear0.max())
print("Linear1:", linear1.dtype, linear1.min(), linear1.max())

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(linear0, cmap='gray')
ax1.imshow(linear1, cmap='gray');

In [ ]:
from skimage import img_as_float, img_as_ubyte

image = data.chelsea()

image_float = img_as_float(image)
image_ubyte = img_as_ubyte(image)

print("type, min, max:", image_float.dtype, image_float.min(), image_float.max())
print("type, min, max:", image_ubyte.dtype, image_ubyte.min(), image_ubyte.max())

print("231/255 =", 231/255.)

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(image_float)
ax1.imshow(image_ubyte);

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 5))
R = image[:,:,0]
axes[0, 0].imshow(R, cmap='jet')
axes[0, 1].imshow(R, cmap='viridis')
axes[1, 0].imshow(R, cmap='magma')
axes[1, 1].imshow(R, cmap='gray');

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(10, 10))
R = image.copy()
G = image.copy()
B = image.copy()

R[:,:,(1,2)] = 0
G[:,:,(0,2)] = 0
B[:,:,(0,1)] = 0
axes[0].imshow(R)
axes[1].imshow(G)
axes[2].imshow(B)


In [ ]:
pwd

In [ ]:
from skimage import io

image = io.imread('../skimage/skimage-tutorials/images/balloon.jpg')

print(type(image))
plt.imshow(image);

In [ ]:
ic = io.imread_collection('../skimage/skimage-tutorials/images/*.png')

print(type(ic), '\n\n', ic)

f, axes = plt.subplots(nrows=1, ncols=len(ic), figsize=(15, 10))

for i, image in enumerate(ic):
    axes[i].imshow(image, cmap='gray')
    axes[i].axis('off')

In [ ]:
img = io.imread('bossier.jpg')
img = img_as_float(img)
print(img.shape, type(img))

In [ ]:
np.array(range(16)).reshape(4,4)

In [ ]:
def pad_image(img):
    ro = img.shape[0]
    co = img.shape[1]
    r = ro + ro % 8
    c = co + co % 8
    if len(img.shape) == 3:
        ch = img.shape[2]
        img2 = np.zeros(r*c*ch).reshape(r,c,ch)
        img2[:ro,:co,:] = img[:ro,:co,:]
    else: # single channel
        img2 = np.zeros(r*c).reshape(r,c)
        img2[:ro,:co] = img[:ro,:co]
    return img2

In [ ]:
img = pad_image(img)
print(img.shape, type(img))

In [ ]:
plt.imshow(img);

In [ ]:
def get_2D_dct(img):
    """ Get 2D Cosine Transform of Image
    """
    d = fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')
    return d # d.astype(int)

def get_2D_idct(coefficients):
    """ Get 2D Inverse Cosine Transform of Image
    """
    id = fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')
    return id # id.astype(int)

def get_8x8_block(img, r_start, c_start, channel):
    """ Get an 8x8 block from the image
    """
    return img[r_start:r_start+8, c_start:c_start+8, channel]

def put_8x8_block(img, block, r_start, c_start, channel):
    img_new = img
    img_new[r_start:r_start+8, c_start:c_start+8, channel] = block
    return img_new

def quantize(dct_block, factor):
    """ Get a quantized version of a matrix
    """
    f = float(factor)
    q = np.floor(dct_block/f)*f
    return q # q.astype(int)

def compress(img, n_coeff):
    img_reconstructed = np.zeros_like(img)

    for start_row in np.arange(0,img.shape[0],8):
        for start_col in np.arange(0,img.shape[1],8):
            for channel in range(0,3):
                block = get_8x8_block(img, start_row, start_col, channel)
                block_dct = get_2D_dct(block)
                # block_dct_quantized = quantize(block_dct, factor)
                block_dct_quantized = get_n_max_vals(block_dct, n_coeff)
                block_reconstructed = get_2D_idct(block_dct_quantized)
                img_reconstructed = put_8x8_block(img_reconstructed, block_reconstructed, start_row, start_col, channel)
    return img_reconstructed

def get_n_max_vals(blk, n):
    # Convert it into a 1D array and take absolute value
    a_1d = np.abs(blk.flatten())
    # Find the indices in the 1D array
    idx_1d = a_1d.argsort()[-n:]
    # Get min of the maxes
    m = a_1d[idx_1d[0]]
    
    b = np.zeros_like(blk)
    b[blk >= m] = blk[blk >= m] # Large positive values
    b[blk <= -m] = blk[blk <= -m] # Large negative values
    return b


In [ ]:
a = np.random.randint(low=0, high=10, size=10).reshape(2, 5) - 5
a_1d = a.flatten()
print(a_1d)
print(a_1d.argsort())
print(a_1d.argsort()[-3:])
print(a)
print(get_n_max_vals(a,3))

In [ ]:
img2 = img_as_ubyte(img)
blk = get_8x8_block(img, 350, 315, 0)
dct = get_2D_dct(blk)
idct = get_2D_idct(dct)

fig, axes = plt.subplots(ncols=3, figsize=(10, 10))

axes[0].imshow(blk, cmap="gray", interpolation="nearest")
axes[1].imshow(dct, cmap="gray", interpolation="nearest")
axes[2].imshow(idct, cmap="gray", interpolation="nearest")


In [ ]:
np.round(dct, decimals=3)

## Converting to YCbCr

https://en.wikipedia.org/wiki/YCbCr

JFIF usage of JPEG allows Y′CbCr where Y′, CB and CR have the full 8-bit range of 0-255:[4]

$$
    \begin{align} Y' &=& 0 &+ (0.299 & \cdot R'_D) &+ (0.587 & \cdot G'_D) &+ (0.114 & \cdot B'_D)\\ C_B &=& 128 & - (0.168736 & \cdot R'_D) &- (0.331264 & \cdot G'_D) &+ (0.5 & \cdot B'_D)\\ C_R &=& 128 &+ (0.5 & \cdot R'_D) &- (0.418688 & \cdot G'_D) &- (0.081312 & \cdot B'_D) \end{align}
$$

And back:

$$
    \begin{align} R &=& Y &&& + 1.402 & \cdot (C_R-128) \\ G &=& Y & - 0.34414 & \cdot (C_B-128)& - 0.71414 & \cdot (C_R-128) \\ B &=& Y & + 1.772 & \cdot (C_B-128)& \end{align} 
$$

Factor out bias terms:

$$
\begin{align}
R &= 1 \times Y + 0 \times (C_B - 128) + 1.402 \times (C_R-128) \\
R &= 1 \times Y + 0 C_B - 0 \times 128 + 1.402 C_R - 1.402 \times 128 \\
R &= - 1.402 \times 128 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
R &= - 179.456 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
\\
G &= 1 \times Y - 0.34414 \times (C_B - 128) - 0.71414 \times (C_R-128) \\
G &= 1 \times Y - 0.34414 \times C_B + 0.34414 \times 128 - 0.71414 \times C_R + 0.71414 \times 128 \\
G &= (0.34414 + 0.71414) \times 128 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
G &= 135.45984 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
\\
B &= 1 \times Y + 1.772 \times (C_B - 128) + 0 \times (C_R-128) \\
B &= 1 \times Y + 1.772 \times C_B - 1.772 \times 128 + 0 \times C_R \\
B &= - 1.772 \times 128 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
B &= - 226.816 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
\end{align}
$$



In [ ]:
def rgb2ycbcr(img):
    image_float = img_as_float(image)
    image_ubyte = img_as_ubyte(img)
    img_new = np.zeros_like(image_ubyte)
    CM_to_ycbcr = np.array([[0, .299, .587, .114],[128, -.168736, -.331264, .5],[128, .5, -.418688, -.081312]])
    
    for row in range(image_ubyte.shape[0]):
        for col in range(image_ubyte.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = image_ubyte[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_ycbcr, pix_with_bias)
    return img_as_float(img_new)

def ycbcr2rgb(img):
    img_new = np.zeros_like(img)
    image_ubyte = img_as_ubyte(img)
    CM_to_rgb = np.array([[-179.456, 1, 0, 1.402],[135.45984, 1, -.34414, -.71414],[-226.816, 1, 1.772, 0]])

    for row in range(image_ubyte.shape[0]):
        for col in range(image_ubyte.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = image_ubyte[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_rgb, pix_with_bias)
    return img_as_float(img_new)

In [ ]:
(0 == 0) & (1 ==1)

In [ ]:
n_coeff = 64

fig, axes = plt.subplots(2, 3, figsize=(15, 15))
for r in range(0,2):
    for c in range(0,3):
        if (r==0) & (c==0): 
            axes[r,c].imshow(img)
        else: 
            img_compressed = compress(img, n_coeff)
            axes[r,c].imshow(img_compressed)
            fname = "b_rgb_" + str(n_coeff) + ".jpg"
            io.imsave(fname, img_as_ubyte(img_compressed))
            # mpimg.imsave(fname, img_compressed)
        n_coeff = n_coeff/2


In [ ]:
img_ycbcr = rgb2ycbcr(img)
img_ycbcr_compressed = compress(img_ycbcr, factor)
img_rgb_compressed = ycbcr2rgb(img_ycbcr_compressed)

imgplot = plt.imshow(img_rgb_compressed)

In [ ]:

fname = "b_ycbcr_" + str(factor) + ".jpg"
mpimg.imsave(fname, img_rgb_compressed)